# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
gb =  GradientBoostingClassifier()

In [6]:
param = {'n_estimators':[100,150],
        'max_depth':[7,11,15],
        'learning_rate': [0.1]}

In [7]:
gscv = GridSearchCV(gb,param,cv = 5,n_jobs = -1)
# here n_jobs = -1 => subsetting of the dataset and parameter setting is done simultaneously
#and not that the decision trees are working independently.
gs_tf_fit = gscv.fit(X_tfidf_feat,data['label'])
pd.DataFrame(gs_tf_fit.cv_results_).sort_values('mean_test_score',ascending =False)[0:5]
#cv_results is messy so we are converting it into a dataframe

C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\deeks\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,279.096508,45.931499,0.318369,0.020156,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965919,0.981132,...,0.970541,0.005422,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,658.724855,32.458348,0.279133,0.009056,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965022,0.973944,...,0.969822,0.003276,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,583.382146,10.300099,0.335119,0.040020,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965919,0.977538,...,0.969463,0.004354,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,527.503050,25.040389,0.304974,0.035207,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965022,0.974843,...,0.969104,0.003229,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,377.479326,10.414966,0.326822,0.009806,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.961435,0.976640,...,0.968385,0.004915,5,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [ ]:
gscv = GridSearchCV(gb,param,cv = 5,n_jobs = -1)
# here n_jobs = -1 => subsetting of the dataset and parameter setting is done simultaneously
#and not that the decision trees are working independently.
gs_cv_fit = gscv.fit(X_count_feat,data['label'])
pd.DataFrame(gs_cv_fit.cv_results_).sort_values('mean_test_score',ascending =False)[0:5]